Langkah 1 - Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2, SelectKBest, RFE
from sklearn.metrics import accuracy_score, classification_report

Langkah 2 - Load Data dan Pengelompokan Data

Load data Titanic-Dataset.csv menggunakan pandas. Selanjutnya, kita akan memisahkan variabel target, yaitu Survived dan mengelompokkan data berdasarkan data numerik dan kategorikal.

In [3]:
# Load Data
df = pd.read_csv("Assets/Titanic-Dataset.csv")

# Pisahkan Survived
y = df["Survived"].astype(int)
X = df.drop(columns=["Survived"])

# Buat list variabel numerik dan kategorikal
# Akan digunakan untuk proses seleksi fitur
# Name tidak akan digunakan karena tidak relevan
num_cols = ["Age", "SibSp", "Parch", "Fare"]
cat_cols = ["Pclass", "Sex", "Embarked"]

Langkah 3 - Ekstraksi Fitur

Pada tahap ini, kita akan melakukan ekstraksi fitur berdasarkan jenis data untuk setiap variabel. Selain itu, kita akan mencoba menggunakan Pipeline dari scikit-learn untuk proses ekstraksi fiturnya. Pipeline merupakan model proses yang dapat kita gunakan berkali kali sehingga memudahkan proses pembuatan model. Stategi ekstraksi fitur yang digunakan adalah imputasi dan penyekalaan. Proses ini juga sebenernya dapat dilakukan pada saat EDA dan proprocessing. Ingat, fitur pada data tabular sebenarnya adalah kolom-kolom dari data tabular.

In [4]:
# Ekstaksi Fitur dengan Pipeline

# Data Numerik
num_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Data Kategorikal
cat_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

Langkah 3.1 - Feature Construction

Selain menggunakan variabel-variabel yang sudah ada, sebetulnya kita dapat membuat fitur baru dalam data tabular berdasarkan proses agregasi atau penggabungan dari variabel-variabel yang lain. Langkah ini disebut sebagai feature construction. 

Kita akan membuat fitur bernama FamiliSize yang merupakan jumlah keluarga yang dapat dihitung berdasarkan SibSp + Parch + 1. SipSp merupakan data terkait dengan jumlah siblings dan spouse. Sedangkan Parch adalah data terkait dengan jumlah orang tua atau anak.

In [5]:
# Buat Fitur FamilySize
X["FamilySize"] = X["SibSp"].fillna(0) + X["Parch"].fillna(0) + 1

# Tambahkan FamilySize pada kelompok numerikal
preprocess = ColumnTransformer([
    ("num", num_tf, num_cols + ["FamilySize"]),
    ("cat", cat_tf, cat_cols),
])

Langkah 4 - Seleksi Fitur

Kita akan melakukan proses seleksi fitur sederhana dengan menggunakan fungsi SelectKBest. Fungsi ini menggunakan metode univariate statistics dalam proses pemilihan fiturnya. Jumlah fitur yang digunakan sesuai dengan jumlah k yang digunakan. 

In [6]:
from sklearn.feature_selection import f_classif
selector_filter = SelectKBest(score_func=f_classif, k=5)

In [7]:
# Seleksi fitur dengan SelectKBest
# Fungsi tersebut akan menggunakan analisis variance
# Baca: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

from sklearn.feature_selection import f_classif
selector_filter = SelectKBest(score_func=f_classif, k=5)

# Buat pipeline final (INGAT INI HANYA PIPELINE, BELUM MEMPROSES DATA)
pipe_filter = Pipeline([
    ("prep", preprocess), # menjalankan pipeline preprocessing
    ("sel", selector_filter), # menjalankan pipeline seleksi fitur
    ("clf", LogisticRegression(max_iter=1000)) # uji dengan model sederhana -> Logistic Regression
])

Langkah 5 - Uji dengan Model

Pada langkah ini kita akan melakukan uji langsung dengan melakukan proses pelatihan menggunakan model Logistic Regression untuk mengetahui hasil dari proses ekstraksi dan seleksi fitur terhadap nilai akurasi.

In [8]:
# Lakukan pelatihan dan uji model
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

pipe_filter.fit(X_train, y_train)
pred = pipe_filter.predict(X_test)
print("=== Filter (ANOVA) + LR ===")
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

=== Filter (ANOVA) + LR ===
Accuracy: 0.776536312849162
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       110
           1       0.73      0.67      0.70        69

    accuracy                           0.78       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.77      0.78      0.77       179



NB: Anda mungkin mendapatkan hasil yang berbeda. 

Berdasarkan informasi tersebut, kita mengetahui tingkat akurasi yang didapatkan sekitar 
0.78
0.78 atau 
78
%
78%. Hasilnya cukup baik namun tidak terlalu baik untuk mengetahui penumpang yang meninggal atau tidak. Hal ini erat kaitannya dengan fitur yang digunakan.

In [9]:
# 1) Nama fitur setelah preprocess
feat_names = pipe_filter.named_steps["prep"].get_feature_names_out()
print("Nama fitur:", feat_names)
print("\n")

# 2) Mask & skor fitur terpilih (SelectKBest)
sel = pipe_filter.named_steps["sel"]
mask = sel.get_support()
selected_names = feat_names[mask]
selected_scores = sel.scores_[mask]
top = sorted(zip(selected_names, selected_scores), key=lambda t: t[1], reverse=True)[:10]
print("Top fitur:", top)

Nama fitur: ['num__Age' 'num__SibSp' 'num__Parch' 'num__Fare' 'num__FamilySize'
 'cat__Pclass_1' 'cat__Pclass_2' 'cat__Pclass_3' 'cat__Sex_female'
 'cat__Sex_male' 'cat__Embarked_C' 'cat__Embarked_Q' 'cat__Embarked_S']


Top fitur: [('cat__Sex_female', np.float64(306.5932488951883)), ('cat__Sex_male', np.float64(306.59324889518797)), ('cat__Pclass_3', np.float64(80.33862734392042)), ('cat__Pclass_1', np.float64(73.99727564291717)), ('num__Fare', np.float64(58.31490728198491))]
